In [215]:
import numpy as np

# from gym.envs.toy_text import BlackjackEnv
import gym
from typing import Literal, List, Tuple, cast, Dict
import plotly.graph_objects as go
from copy import deepcopy
import math
import sys


import io


In [216]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [217]:

np.random.seed(0)
env = gym.make('gym_cliffwalking:cliffwalking-v0')
env.seed(0)

obs = env.reset()


In [218]:
obs

0

In [219]:
Position = Literal[
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
]
State = Position
Observation = Position

# 0: Right; 1: Down; 2: Left; 3: Up
Action = Literal[0, 1, 2, 3]
Cols = Literal[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Rows = Literal[0, 1, 2, 3]
Coordination = Tuple[Rows, Cols]
Reward = float
Step = Tuple[State, Action, Reward]
Episode = List[Step]


In [220]:
all_states = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    31,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    39,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
]

all_actions = [0, 1, 2, 3]
allowed_action: List[List[Action]] = [[0, 1, 2, 3]
                                      for _ in range(len(all_states))]
cols_per_row = 12
nums_of_state = len(all_states)
nums_of_state_action_pair = len(all_actions) * len(all_states)
terminal_state: Literal[11] = 11


def get_state_from_pos(pos: Position) -> State:
    assert pos in all_states, "wrong state encountered"
    return pos


def get_pos_from_state(s: State) -> Position:
    assert s in all_states, "wrong state encountered"
    return s


def get_state_from_cor(cor: Coordination) -> State:
    row, col = cor
    return row * cols_per_row + col


def get_cor_from_state(s: State) -> Coordination:
    row = s // cols_per_row
    col = s % cols_per_row
    return (row, col)


def get_idx_from_state(s: State) -> Position:
    assert s in all_states, "wrong state encountered"
    return s


def get_state_from_idx(idx: int) -> State:
    s = cast(State, idx)
    assert s in all_states, "wrong state encountered"
    return s


def get_idx_from_state_action(state: State, action: Action) -> Position:
    assert state in all_states, "wrong state encountered"
    assert action in all_actions, "wrong action encountered"

    return cast(Position, state * len(all_actions) + action)


def get_state_action_from_idx(idx: Position) -> Tuple[State, Action]:
    state = idx // len(all_states)
    action = idx % len(all_states)

    assert state in all_states, "wrong state encountered"
    assert action in all_actions, "wrong action encountered"

    return (cast(State, state), cast(Action, action))


In [221]:
class Agent:
    def __init__(self, env: gym.Env, alpha: float, sigma: float):
        self.env = env
        self.alpha = alpha
        self.sigma = sigma

        self.clear()

    def take_action(self, s: State) -> Action:
        # act = self.pai[get_idx_from_state(s)]
        act = np.random.choice(all_actions)
        return act

    def random_action(self, s: State) -> Dict[Action, float]:
        if s == 23:
            return {
                cast(Action, "r"): 0.5,
                cast(Action, "d"): 0.5,
            }
        return {
            cast(Action, "r"): 0.25,
            cast(Action, "d"): 0.25,
            cast(Action, "l"): 0.25,
            cast(Action, "u"): 0.25,
        }

    def dynamic(self, s: State, a: Action) -> Dict[Tuple[State, Reward], float]:
        if s == 0:
            # case 0:
            if a == "r":
                # case "r":
                return {(0, -100): 1}
            elif a == "d":  # case "d":
                return {(0, -1): 1}
            elif a == "l":  # case "l":
                return {(0, -1): 1}
            elif a == "u":  # case "u":
                return {(12, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 12:
        # elif s >= 1 and s <= 11:
        #     if a == "r":
        #         return {(0, -99): 1}
        elif s == 12:
            # match a:

            # case "r":
            if a == "r":
                return {(13, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -1): 1}
            elif a == "l":
                #    case "l":
                return {(12, -1): 1}
                # case "u":
            elif a == "u":
                return {(24, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 13:
        elif s == 13:
            # match a:
            if a == "r":
                # case "r":
                return {(14, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(12, -1): 1}
                # case "u":
            elif a == "u":
                return {(25, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 14:
        elif s == 14:
            # match a:
            if a == "r":
                # case "r":
                return {(15, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(13, -1): 1}
                # case "u":
            elif a == "u":
                return {(26, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")

            # case 15:
        elif s == 15:
            # match a:
            if a == "r":
                # case "r":
                return {(16, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(14, -1): 1}
                # case "u":
            elif a == "u":
                return {(27, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 16:
            # case 16:
            # match a:
            if a == "r":
                # case "r":
                return {(17, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(15, -1): 1}
                # case "u":
            elif a == "u":
                return {(28, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 17:
        elif s == 17:
            # match a:
            if a == "r":
                # case "r":
                return {(18, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(16, -1): 1}
                # case "u":
            elif a == "u":
                return {(29, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 18:
            # case 18:
            # match a:

            # case "r":
            if a == "r":
                return {(19, -1): 1}
            # case "d":
            elif a == "d":
                return {(0, -100): 1}
            # case "l":
            elif a == "l":
                return {(17, -1): 1}
            # case "u":
            elif a == "u":
                return {(30, -1): 1}
            # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 19:
        elif s == 19:
            # match a:
            # case "r":
            if a == "r":
                return {(20, -1): 1}
            # case "d":
            elif a == "d":
                return {(0, -100): 1}
            # case "l":
            elif a == "l":
                return {(18, -1): 1}
            # case "u":
            elif a == "u":
                return {(31, -1): 1}
            # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 20:
            # case 20:
            # match a:
            if a == "r":
                # case "r":
                return {(21, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(19, -1): 1}
                # case "u":
            elif a == "u":
                return {(32, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 21:
            # case 21:
            # match a:
            if a == "r":
                # case "r":
                return {(22, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(20, -1): 1}
                # case "u":
            elif a == "u":
                return {(33, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 22:

            # case 22:
            # match a:
            if a == "r":
                # case "r":
                return {(23, -1): 1}
                # case "d":
            elif a == "d":
                return {(0, -100): 1}
                # case "l":
            elif a == "l":
                return {(21, -1): 1}
                # case "u":
            elif a == "u":
                return {(34, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 23:
        elif s == 23:
            # match a:
            if a == "r":
                # case "r":
                return {(23, -1): 1}
                # case "d":
            elif a == "d":
                return {(11, -1): 1}
                # case "l":
            elif a == "l":
                return {(22, -1): 1}
                # case "u":
            elif a == "u":
                return {(35, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 24:
            # case 24:
            # match a:
            if a == "r":
                # case "r":
                return {(25, -1): 1}
                # case "d":
            elif a == "d":
                return {(12, -1): 1}
                # case "l":
            elif a == "l":
                return {(24, -1): 1}
                # case "u":
            elif a == "u":
                return {(36, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 25:
            # case 25:
            # match a:
            if a == "r":
                # case "r":
                return {(26, -1): 1}
                # case "d":
            elif a == "d":
                return {(13, -1): 1}
                # case "l":
            elif a == "l":
                return {(24, -1): 1}
                # case "u":
            elif a == "u":
                return {(37, -1): 1}
            # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 26:
        elif s == 26:
            # match a:
            if a == "r":
                # case "r":
                return {(27, -1): 1}
                # case "d":
            elif a == "d":
                return {(14, -1): 1}
                # case "l":
            elif a == "l":
                return {(25, -1): 1}
                # case "u":
            elif a == "u":
                return {(38, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 27:
            # case 27:
            # match a:
            if a == "r":
                # case "r":
                return {(28, -1): 1}
                # case "d":
            elif a == "d":
                return {(15, -1): 1}
                # case "l":
            elif a == "l":
                return {(26, -1): 1}
                # case "u":
            elif a == "u":
                return {(39, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 28:
            # case 28:
            # match a:
            if a == "r":
                # case "r":
                return {(29, -1): 1}
                # case "d":
            elif a == "d":
                return {(16, -1): 1}
                # case "l":
            elif a == "l":
                return {(27, -1): 1}
                # case "u":
            elif a == "u":
                return {(40, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 29:
            # case 29:

            # math a:
            if a == "r":
                # case "r":
                return {(30, -1): 1}
                # case "d":
            elif a == "d":
                return {(17, -1): 1}
                # case "l":
            elif a == "l":
                return {(28, -1): 1}
                # case "u":
            elif a == "u":
                return {(41, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")

            # case 30:
        elif s == 30:
            # match a:
            # case "r":
            if a == "r":
                return {(31, -1): 1}
                # case "d":
            elif a == "d":
                return {(18, -1): 1}
                # case "l":
            elif a == "l":
                return {(29, -1): 1}
                # case "u":
            elif a == "u":
                return {(42, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 31:
            # case 31:
            # match a:
            if a == "r":

                # case "r":
                return {(32, -1): 1}
                # case "d":
            elif a == "d":
                return {(19, -1): 1}
                # case "l":
            elif a == "l":
                return {(30, -1): 1}
                # case "u":
            elif a == "u":
                return {(43, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 32:
            # case 32:
            # match a:
            if a == "r":
                # case "r":
                return {(33, -1): 1}
                # case "d":
            elif a == "d":
                return {(20, -1): 1}
                # case "l":
            elif a == "l":
                return {(31, -1): 1}
                # case "u":
            elif a == "u":
                return {(44, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 33:
            # case 33:
            # match a:
            # if
            # case "r":
            if a == "r":
                return {(34, -1): 1}
                # case "d":
            elif a == "d":
                return {(21, -1): 1}
                # case "l":
            elif a == "l":
                return {(32, -1): 1}
                # case "u":
            elif a == "u":
                return {(45, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 34:
            # case 34:
            # match a:
            if a == "r":
                # case "r":
                return {(35, -1): 1}
                # case "d":
            elif a == "d":
                return {(22, -1): 1}
            elif a == "l":
                # case "l":
                return {(33, -1): 1}
                # case "u":
            elif a == "u":
                return {(46, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 35:
            # case 35:
            # match a:
            if a == "r":
                # case "r":
                return {(35, -1): 1}
                # case "d":
            elif a == "d":

                return {(23, -1): 1}
                # case "l":
            elif a == "l":
                return {(34, -1): 1}
                # case "u":
            elif a == "u":
                return {(47, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 36:
            # case 36:
            # match a:
            if a == "r":
                # case "r":
                return {(37, -1): 1}
                # case "d":
            elif a == "d":
                return {(24, -1): 1}
                # case "l":
            elif a == "l":
                return {(36, -1): 1}
                # case "u":
            elif a == "u":
                return {(36, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 37:

            # case 37:
            # match a:
            if a == "r":
                # case "r":
                return {(38, -1): 1}
                # case "d":
            elif a == "d":
                return {(25, -1): 1}
                # case "l":
            elif a == "l":
                return {(36, -1): 1}
                # case "u":
            elif a == "u":
                return {(37, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 38:
            # case 38:
            # match a:
            if a == "r":
                # case "r":
                return {(39, -1): 1}
                # case "d":
            elif a == "d":
                return {(26, -1): 1}
                # case "l":
            elif a == "l":
                return {(37, -1): 1}
                # case "u":
            elif a == "u":
                return {(38, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 39:
        elif s == 39:
            # match a:
            if a == "r":
                # case "r":
                return {(40, -1): 1}
                # case "d":
            elif a == "d":
                return {(27, -1): 1}
                # case "l":
            elif a == "l":
                return {(38, -1): 1}
                # case "u":
            elif a == "u":
                return {(39, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 40:
            # case 40:
            # match a:
            if a == "r":
                # case "r":
                return {(41, -1): 1}
                # case "d":
            elif a == "d":
                return {(28, -1): 1}
                # case "l":
            elif a == "l":
                return {(39, -1): 1}
                # case "u":
            elif a == "u":
                return {(40, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
            # case 41:
        elif s == 41:
            # match a:
            if a == "r":
                # case "r":
                return {(42, -1): 1}
            elif a == "d":
                # case "d":
                return {(29, -1): 1}
                # case "l":
            elif a == "l":
                return {(40, -1): 1}
                # case "u":
            elif a == "u":
                return {(41, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 42:
            # case 42:
            # match a:
            if a == "r":
                # case "r":
                return {(43, -1): 1}
                # case "d":
            elif a == "d":
                return {(30, -1): 1}
                # case "l":
            elif a == "l":
                return {(41, -1): 1}
                # case "u":
            elif a == "u":
                return {(42, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 43:
            # case 43:
            # match a:
            if a == "r":
                # case "r":
                return {(44, -1): 1}
            elif a == "d":
                # case "d":
                return {(31, -1): 1}
                # case "l":
            elif a == "l":
                return {(42, -1): 1}
                # case "u":
            elif a == "u":
                return {(43, -1): 1}
            else:
                # case _:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 44:
            # case 44:
            # match a:
            if a == "r":
                # case "r":
                return {(45, -1): 1}
            elif a == "d":
                # case "d":
                return {(32, -1): 1}
            elif a == "l":
                # case "l":
                return {(43, -1): 1}
            elif a == "u":
                # case "u":
                return {(44, -1): 1}
            else:
                # case _:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 45:
            # case 45:
            # match a:
            if a == "r":
                # case "r":
                return {(46, -1): 1}
            elif a == "d":
                # case "d":
                return {(33, -1): 1}
            elif a == "l":
                # case "l":
                return {(44, -1): 1}
                # case "u":
            elif a == "u":
                return {(45, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 46:
            # case 46:
            # match a:
            if a == "r":
                # case "r":
                return {(47, -1): 1}
            elif a == "d":
                # case "d":

                return {(34, -1): 1}
                # case "l":
            elif a == "l":
                return {(45, -1): 1}
                # case "u":
            elif a == "u":
                return {(46, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        elif s == 47:
            # case 47:
            # match a:
            if a == "r":
                # case "r":
                return {(47, -1): 1}
                # case "d":
            elif a == "d":
                return {(35, -1): 1}
                # case "l":
            elif a == "l":
                return {(46, -1): 1}
                # case "u":
            elif a == "u":
                return {(47, -1): 1}
                # case _:
            else:
                raise ValueError(
                    f"unexpected action encountered in dynamic: {a}")
        else:
            raise ValueError(f"unexpected state encountered in dynamic: {s}")

    def reset(self):
        self.end: bool = False

        pos = self.env.reset()
        self.current_state: State = get_state_from_pos(pos)
        self.current_episode: Episode = []

    def dp_evaluate(self, gamma: float = 1.0, delta: float = 1e-2) -> List[float]:
        v = cast(List[float], np.random.random(size=nums_of_state))
        v[get_idx_from_state(11)] = 0
        while True:
            _delta = 0
            for idx, old_v in enumerate(v):

                if idx >= 1 and idx < 12:
                    continue

                state = get_state_from_idx(idx)
                old_v = v[state]
                v[state] = np.sum(
                    [
                        prob
                        * np.sum(
                            [
                                p *
                                (rwd + gamma * (v[next_state] if (next_state ==
                                 0 or next_state >= 11) else v[0] - 100))
                                for ((next_state, rwd), p) in self.dynamic(
                                    state, act
                                ).items()
                            ]
                        )
                        for (act, prob) in self.random_action(state).items()
                    ]
                )

                _delta = np.max([_delta, np.abs(v[state] - old_v)])
            if _delta < delta:
                break
        return v

    def clear(self):
        self.reset()

        # self.pai: List[Action] = [
        #     np.random.choice(all_actions) for _ in range(len(all_states))
        # ]

        # self.random_pai: List[Action] = []

        self.V = cast(List[float], np.random.random(size=nums_of_state))

        self.V[get_idx_from_state(get_state_from_cor((0, 11)))] = 0
        self.Q = cast(List[float], np.random.random(
            size=nums_of_state_action_pair))
        for act in allowed_action[terminal_state]:
            self.Q[get_idx_from_state_action(terminal_state, act)] = 0

    def close(self):
        self.env.close()

    def step(self) -> Tuple[Observation, bool]:
        assert not self.end, "cannot step on a ended agent"

        act = self.take_action(self.current_state)
        (obs, reward, done, info) = self.env.step(act)

        self.current_episode.append((self.current_state, act, reward))

        obs = cast(Observation, obs)
        self.current_state = obs

        if done:
            self.end = True
            return obs, True
        return obs, False


In [222]:
TOTAL_RUNS = 1000

agent = Agent(env, 1, 0.5)
# agent.reset()

# for i in range(TOTAL_RUNS):
#     start_pos = agent.reset()

#     while True:
#         (obs, done) = agent.step()

#         if done:
#             break

# agent.dp_evaluate()

agent.close()


In [223]:
def print_state_value_table(stateValues: List[float]):
    output = io.StringIO()
    for i, v in enumerate(stateValues):
        print("{:8.0f}".format(v if i == 0 or i >=
              11 else float('nan')), end=" ", file=output)
        if (i+1) % 12 == 0:
            # print("\033[F")
            # sys
            print(file=output)
    # print('\n'.join(output.getvalue().split('\n').reverse()))
    outStr = output.getvalue()
    # outStr.split('\n')
    print('\n'.join(reversed(outStr.split('\n'))))


In [224]:

V = agent.dp_evaluate()

In [225]:
# get_cor_from_pos(get_pos_from_state(cast(State, 23)))

print_state_value_table(V)



  -33765   -33699   -33547   -33276   -32833   -32132   -31044   -29384   -26919   -23448   -19113   -15208 
  -33827   -33780   -33662   -33445   -33087   -32515   -31612   -30185   -27920   -24307   -18681   -11298 
  -33932   -33930   -33870   -33751   -33550   -33226   -32699   -31822   -30264   -27177   -20000       -2 
  -34035      nan      nan      nan      nan      nan      nan      nan      nan      nan      nan        0 
